<a href="https://colab.research.google.com/github/Auxilus08/Compiler-Design/blob/main/CD_Prac_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
class ThreeAddressCodeGenerator:
    def __init__(self):
        self.label_counter = 0
        self.temp_counter = 0
        self.code = []

    def new_label(self):
        self.label_counter += 1  # generate the unique labels
        return f"L{self.label_counter}"

    def new_temp(self):
        self.temp_counter += 1  # generate the temporary labels
        return f"T{self.temp_counter}"

    def generate_code(self, ast):  # traverse the AST and call the appropriate mode based on node type
        if isinstance(ast, list):
            for node in ast:
                self.generate_code(node)
        elif ast["type"] == "if_then_else":
            self.generate_if_then_else(ast)
        elif ast["type"] == "assignment":
            self.generate_assignment(ast)
        elif ast["type"] == "expression":
            return self.generate_expression(ast)
        elif ast["type"] == "condition":
            return self.generate_condition(ast)
        else:
            raise ValueError(f"Unsupported AST type: {ast['type']}")

    def generate_condition(self, ast): # handle genertion of code for a condition node in AST.
        return ast["text"] # Returning the text of the condition for simplicity

    def generate_assignment(self, ast):  # generate TAC  for assignment statement
        target = ast["target"]
        source = self.generate_code(ast["source"])
        self.code.append(f"{target} = {source}")


    def generate_expression(self, ast): # generate TAC for an expression node
        if ast["type"] == "expression":
            if "op" in ast:
                left = self.generate_code(ast["left"])
                right = self.generate_code(ast["right"])
                temp = self.new_temp()
                self.code.append(f"{temp} = {left} {ast['op']} {right}")
                return temp
            elif "value" in ast:
                return ast["value"]
            elif "variable" in ast:
                return ast["variable"]
            else:
                raise ValueError(f"Unsupported expression type: {ast}")
        else:
             # If the node is not an expression, assume it's a variable or literal
             if "value" in ast:
                 return ast["value"]
             elif "variable" in ast:
                 return ast["variable"]
             else:
                raise ValueError(f"Unsupported expression type: {ast}")


    def generate_if_then_else(self, ast): # generate TAC for an if-then-else statement
        false_label = self.new_label()
        end_label = self.new_label()

        condition_code = self.generate_code(ast["condition"])
        self.code.append(f"if ({condition_code}) GOTO {self.label_counter + 1}") # GOTO next instruction if true
        self.code.append(f"GOTO {false_label}")

        # Then part
        self.generate_code(ast["then_part"])
        self.code.append(f"GOTO {end_label}")

        # Else part
        self.code.append(f"{false_label}:")
        self.generate_code(ast["else_part"])

        self.code.append(f"{end_label}:")


ast = {
    "type": "if_then_else",
    "condition": {
        "type": "condition",
        "text": "a < 5"
    },
    "then_part": [
        {
            "type": "assignment",
            "target": "c",
            "source": {
                "type": "expression",
                "op": "+",
                "left": {"type": "expression", "variable": "b"},
                "right": {"type": "expression", "variable": "d"}
            }
        },
        {
            "type": "assignment",
            "target": "d",
            "source": {
                "type": "expression",
                "op": "+",
                "left": {"type": "expression", "variable": "i"},
                "right": {"type": "expression", "variable": "j"}
            }
        }
    ],
    "else_part": [
        {
            "type": "assignment",
            "target": "d",
            "source": {
                "type": "expression",
                "op": "+",
                "left": {"type": "expression", "variable": "a"},
                "right": {"type": "expression", "variable": "b"}
            }
        },
        {
            "type": "assignment",
            "target": "k",
            "source": {
                "type": "expression",
                "op": "+",
                "left": {"type": "expression", "variable": "x"},
                "right": {"type": "expression", "variable": "y"}
            }
        }
    ]
}


generator = ThreeAddressCodeGenerator()
generator.generate_code(ast)

generator.code.append("END")


numbered_code = [f"{i+1}) {instruction}" for i, instruction in enumerate(generator.code)]

print("\n".join(numbered_code))

1) if (a < 5) GOTO 3
2) GOTO L1
3) T1 = b + d
4) c = T1
5) T2 = i + j
6) d = T2
7) GOTO L2
8) L1:
9) T3 = a + b
10) d = T3
11) T4 = x + y
12) k = T4
13) L2:
14) END
